In [1]:
import pickle
import pandas as pd
import numpy as np

from astropy.io import ascii
from astropy.coordinates import SkyCoord
from SPLASH.pipeline import Splash_Pipeline

# Load in data

In [2]:
# Load in bts data
bts_hosts = ascii.read('/Users/adamboesky/Research/ay98/clean_data/bts_hosts.ecsv')
bts_df = bts_hosts.to_pandas()
bts_df.head()

objID_3pi   raStack_3pi decStack_3pi primaryDetection_3pi  \
0  126681757766820931  175.77662082  15.56701789                    1   
1  148231465149294617  146.51493972  33.52836114                    1   
2  153731349546771375  134.95471695  38.10897029                    1   
3  169101206490177628  120.64903126   50.9224606                    1   
4  159661026075384557  102.60753619  43.05326583                    1   

  gKronMag_3pi rKronMag_3pi iKronMag_3pi zKronMag_3pi yKronMag_3pi  \
0      16.0256      15.2881      14.9533      14.7114      14.5005   
1      18.2387      17.9746       17.875      17.8532      17.7821   
2      18.7478      19.0585      18.6466      18.8301      18.6221   
3      16.5325      15.6522      15.2551      15.0064      14.7582   
4          nan      14.2309       13.166      12.9816      12.8047   

  gKronMagErr_3pi rKronMagErr_3pi iKronMagErr_3pi zKronMagErr_3pi  \
0        0.002978        0.001497        0.001272        0.001197   
1        0.004157        0.004871        0.004281        0.014407   
2        0.006392        0.009475        0.004919        0.017235   
3        0.002335        0.002128        0.000867        0.001623   
4             nan        0.000836        0.000408        0.001003   

  yKronMagErr_3pi ps_score_3pi       SN_ra     SN_dec sn_class  sn_redshift  \
0        0.002235    0.1110298  175.776542  15.567139      NaN          NaN   
1        0.019814     0.021125  146.514792  33.528250    SN II      0.03800   
2        0.038667   0.01741071  134.954667  38.109056    SN II      0.07247   
3        0.003512   0.07590476  120.648958  50.922528    SN Ia      0.05295   
4        0.001687    0.1546488  102.607542  43.053222   SN IIn      0.01885   

          ZTFID  
0           NaN  
1  ZTF18aacemcn  
2  ZTF18aacnlxz  
3  ZTF18aadlaxo  
4  ZTF18aadmssd

In [12]:
def get_angular_separation(ra1, dec1, ra2, dec2, unit1='deg', unit2='deg'):
    """Function to calculate angular separation in arcseconds"""
    coord1 = SkyCoord(ra1, dec1, unit=unit1)
    coord2 = SkyCoord(ra2, dec2, unit=unit2)
    return coord1.separation(coord2).arcsec

def ab_mag_to_flux(AB_mag: np.ndarray) -> np.ndarray:
    """Convert AB magnitude to flux in mJy"""
    return 10**((AB_mag - 8.9) / -2.5) * 1000

ab_magerr_to_ferr = lambda sigma_m, f: np.abs(f * np.log(10) * (sigma_m / 2.5))  # transformation on the error of a magnitude turned into flux

# Add angular separation and grib data
bts_df['angular_separation_arcsec'] = get_angular_separation(bts_df['raStack_3pi'], bts_df['decStack_3pi'], bts_df['SN_ra'], bts_df['SN_dec'], unit1='deg', unit2='deg')
grizy = bts_df[['gKronMag_3pi', 'rKronMag_3pi', 'iKronMag_3pi', 'zKronMag_3pi', 'yKronMag_3pi']].to_numpy().astype(float)
grizy_err = bts_df[['gKronMagErr_3pi', 'rKronMagErr_3pi', 'iKronMagErr_3pi', 'zKronMagErr_3pi', 'yKronMagErr_3pi']].to_numpy().astype(float)
angular_seps = bts_df['angular_separation_arcsec'].to_numpy().astype(float)

# Convert the grizy data to mJy
grizy = ab_mag_to_flux(grizy)
grizy_err = ab_magerr_to_ferr(grizy_err, grizy)

grizy, angular_seps

(array([[ 1.41175716,  2.78458193,  3.79035598,  4.73630869,  5.75174998],
        [ 0.18387386,  0.23450926,  0.25703958,  0.26225272,  0.28000127],
        [ 0.11504825,  0.08641716,  0.12628739,  0.10664979,  0.12916951],
        ...,
        [ 3.76946747,  6.12801754,  7.62500267,  9.22486459, 10.88930093],
        [ 1.04327788, 34.86582934, 40.15318502, 12.13053625, 24.01928084],
        [ 1.29741824,  3.02691343,  4.44344695,  6.23907204,  5.06944015]]),
 array([ 0.51487243,  0.59790783,  0.33839217, ...,  0.80115689,
        26.58843375,  6.92645833]))

In [13]:
nan_mask = ~(np.sum(np.isnan(grizy), axis=1)==0)

# Use the SPLASH pipeline

In [14]:
import importlib
import SPLASH.pipeline
importlib.reload(SPLASH.pipeline)
from SPLASH.pipeline import Splash_Pipeline

# Load pipeline object
pipeline = Splash_Pipeline(pipeline_version='weighted_full_band',   # the default version of the pipeline
                           pre_transformed=False,                   # whether the given data is pre-logged and nnormalized
                           within_4sigma=True,                      # whether we only want to classify objects with properties within 4-sigma of the training set
                           nan_thresh_ratio=1.0)                    # to keep this notebook concise, we are allowing the pipeline to imput any num of nans

In [15]:
# Predict the classes. n_resamples is the number of boostraps for getting the median predicted host properties.
classes = pipeline.predict_classes(grizy, angular_seps, grizy_err, n_resamples=50)

[3.83505654 3.67871757 3.96292865 3.61150645 3.47098468] [1.16519241 1.09036387 1.1316336  1.08615098 1.04548113]


In [16]:
print(f'Number of each class:\n{pd.Series(classes).value_counts()}\n\nThe class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma.')

Number of each class:
 0    3921
 4     375
 1     156
 3      40
 2       3
-1       2
Name: count, dtype: int64

The class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma.


In [17]:
# We can also take a look at the predicted host properties of each supernova
props, props_err = pipeline.predict_host_properties(grizy, grizy_err, n_resamples=50, return_normalized=False)  # return_normalized will return the properties normalized with the train mean and std
props, props_err  # in order (log(mass), log(sfr), redshift)

[3.83505654 3.67871757 3.96292865 3.61150645 3.47098468] [1.16519241 1.09036387 1.1316336  1.08615098 1.04548113]


(array([[11.03105525, -0.37596385,  0.09514158],
        [ 9.57517772, -0.7333718 ,  0.10061707],
        [ 8.96926393, -0.79599632,  0.09335136],
        ...,
        [10.75070607, -0.04103815,  0.03659505],
        [10.91012792, -2.31738271, -0.06197032],
        [10.98826124, -2.10289793,  0.10901751]]),
 array([[4.84805576e-03, 1.13760916e-03, 3.56388281e-04],
        [4.91402535e-02, 8.66192764e-02, 5.97183284e-03],
        [6.55539605e-02, 1.31478993e-01, 8.16184667e-03],
        ...,
        [3.69723008e-03, 4.42506516e-04, 3.71092049e-04],
        [7.33016208e-04, 3.32646862e-04, 2.58938597e-05],
        [2.45852021e-03, 7.63355354e-02, 1.14955794e-03]]))